In [ ]:
!pip install joblib
!pip install pykalman
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import joblib
import json
import math
import pykalman
from datetime import datetime
import matplotlib.pyplot as plt


#read data files

data_source = pd.read_pickle('Data/Final data/data_source.pkl')
data_target = pd.read_pickle('Data/Final data/data_target.pkl')
data_target_2 = pd.read_pickle('Data/Final data/data_target_2.pkl')


#Function that checks for NaN values in each column

def check_nan_intervals(df,mode):
    for column in df.columns:

        # Get a boolean series indicating where NaN values are
        nan_series = df[column].isna()

        # Find the indices where NaN values start and end
        nan_starts = nan_series[(nan_series != nan_series.shift(1)) & nan_series].index
        nan_ends = nan_series[(nan_series != nan_series.shift(-1)) & nan_series].index

        # Print the intervals
        longest_NaN_streak = 0
        for start, end in zip(nan_starts, nan_ends):
            if (longest_NaN_streak < end-start+1):
                longest_NaN_streak = end-start+1
            if (mode == 'NaN ANALYTICS'):
              print(f"Column: {column}")
              print(f"NaN values from row {start} to {end}")

        # Check if there are trailing NaNs till the end of the dataframe
        if nan_series.iloc[-1]:
            if (longest_NaN_streak < len(df) - nan_starts[-1]):
              longest_NaN_streak = len(df) - nan_starts[-1]
            if (mode == 'NaN ANALYTICS'):
              print(f"NaN values from row {nan_starts[-1]} to the end")

        if (mode == 'NaN STREAK'):
            if longest_NaN_streak > 0:
                print(f"Column: {column}")
                print(longest_NaN_streak)
        # Extra line for better readability
        print()


#Function that imputates NaN values in a dataframe's column based on NaN values spread

def check_longest_nan_streak_and_imputate(df,dataset):
    for column in df.columns:

        if (column != 'Electrical_Consumption' and dataset == 'source'):
            print(f"Column: {column}")

            # Get a boolean series indicating where NaN values are
            nan_series = df[column].isna()

            # Find the indices where NaN values start and end
            nan_starts = nan_series[(nan_series != nan_series.shift(1)) & nan_series].index
            nan_ends = nan_series[(nan_series != nan_series.shift(-1)) & nan_series].index

            # Print the intervals
            longest_NaN_streak = 0
            for start, end in zip(nan_starts, nan_ends):
                if (longest_NaN_streak < end - start + 1):
                    longest_NaN_streak = end - start + 1

            # Check if there are trailing NaNs till the end of the dataframe
            if nan_series.iloc[-1]:
                if (longest_NaN_streak < len(df) - nan_starts[-1]+1):
                  longest_NaN_streak = len(df) - nan_starts[-1]+1

            print(longest_NaN_streak)

            if longest_NaN_streak > 0:

                mov_avg_window = longest_NaN_streak + 2

                col_data = df[column].copy()

                # Apply moving average imputation
                imputed_col = col_data.rolling(window=mov_avg_window, min_periods=1).mean()

                # Fill NaN values with the computed moving average
                col_data.fillna(imputed_col, inplace=True)

                col_data.fillna(method='bfill', inplace=True)

                # Update the original DataFrame with the imputed values
                df[column] = col_data

                print(f"Column : {column} is imputated")

                # Extra line for better readability
                print()

        elif (column == 'Electrical_Consumption' and (dataset == 'source' or dataset == 'target')):
            print(f"Column: {column}")

            # Get a boolean series indicating where NaN values are
            nan_series = df[column].isna()

            # Find the indices where NaN values start and end
            nan_starts = nan_series[(nan_series != nan_series.shift(1)) & nan_series].index
            nan_ends = nan_series[(nan_series != nan_series.shift(-1)) & nan_series].index

            # Print the intervals
            longest_NaN_streak = 0
            for start, end in zip(nan_starts, nan_ends):
                if (longest_NaN_streak < end - start + 1):
                    longest_NaN_streak = end - start + 1

            # Check if there are trailing NaNs till the end of the dataframe
            if nan_series.iloc[-1]:
                if (longest_NaN_streak < len(df) - nan_starts[-1]+1):
                  longest_NaN_streak = len(df) - nan_starts[-1]+1

            print(longest_NaN_streak)

            if longest_NaN_streak > 0:
                for i in range(len(df)):
                    if pd.isnull(df.at[i, column]):
                        value = 'NaN'
                        back = i
                        front = i
                        while (value != 'Imputated'):
                            if (front+24 < len(df)):
                                front = front+24
                                if pd.notnull(df.at[front, column]):
                                    df.at[i, column] = df.at[front, column]
                                    value = 'Imputated'
                            elif (back-24 > -1):
                                back = back-24
                                if pd.notnull(df.at[back, column]):
                                    df.at[i, column] = df.at[back, column]
                                    value = 'Imputated'
                print(f"Column : {column} is imputated")

                # Extra line for better readability
                print()

        elif (column != 'CloudCoverage' and (dataset == 'target' or dataset == 'target_2')):
            print(f"Column: {column}")

            # Get a boolean series indicating where NaN values are
            nan_series = df[column].isna()

            # Find the indices where NaN values start and end
            nan_starts = nan_series[(nan_series != nan_series.shift(1)) & nan_series].index
            nan_ends = nan_series[(nan_series != nan_series.shift(-1)) & nan_series].index

            # Print the intervals
            longest_NaN_streak = 0
            for start, end in zip(nan_starts, nan_ends):
                if (longest_NaN_streak < end - start + 1):
                    longest_NaN_streak = end - start + 1

            # Check if there are trailing NaNs till the end of the dataframe
            if nan_series.iloc[-1]:
                if (longest_NaN_streak < len(df) - nan_starts[-1]+1):
                  longest_NaN_streak = len(df) - nan_starts[-1]+1

            print(longest_NaN_streak)

            if longest_NaN_streak > 0:

                mov_avg_window = longest_NaN_streak + 2

                col_data = df[column].copy()

                # Apply moving average imputation
                imputed_col = col_data.rolling(window=mov_avg_window, min_periods=1).mean()

                # Fill NaN values with the computed moving average
                col_data.fillna(imputed_col, inplace=True)

                col_data.fillna(method='bfill', inplace=True)

                # Update the original DataFrame with the imputed values
                df[column] = col_data

                print(f"Column : {column} is imputated")

                # Extra line for better readability
                print()

        elif (column == 'CloudCoverage' and (dataset == 'target' or dataset == 'target_2')):

            df[column] = df[column].interpolate(method='spline', order=3)

            print(f"Column : {column} is imputated")

            # Extra line for better readability
            print()


#Extract month information from timestamp column and perform one-hot encoding

def timestamp_month_processing(data):

    data['timestamp'] = data['timestamp'].dt.hour

    # Get the dummies for column 'month'
    dummies = pd.get_dummies(data['month'], prefix='month')

    # Find the original column position
    col_position = data.columns.get_loc('month')

    # Drop the original column 'month'
    data.drop('month', axis=1, inplace=True)

    # Insert the dummy columns at the original position
    for i, col in enumerate(dummies.columns):
        data.insert(col_position + i, col, dummies[col])

    month_cols = [ 'month_1',
                   'month_2',
                   'month_3',
                   'month_4',
                   'month_5',
                   'month_6',
                   'month_7',
                   'month_8',
                   'month_9',
                   'month_10',
                   'month_11',
                   'month_12']

    for col in month_cols:
        data[col] = data[col].astype(int)


#train test split function

def train_test_split(data):

    data = data[cols]
    train = data.iloc[:int(len(data)*split),:]
    test = data.iloc[int(len(data)*split):,]
    return train,test

def strip_leading_zeros(x):
    return int(x.lstrip('0')) if isinstance(x, str) else x

#Function that creates the sequential input from the dataset provided

def split_sequences(sequences, n_steps, n_outputs):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix < len(sequences):
            # gather input and output parts of the pattern

            if (len(sequences.iloc[i:end_ix, :]) == 48 and len(sequences.iloc[end_ix:(end_ix+n_outputs), -1]) == 1):
                seq_x = sequences.iloc[i:end_ix, :]
                seq_y = sequences.iloc[end_ix:(end_ix+n_outputs), -1]
            else :
                #debugging prints
                print(i)
                print(sequences.iloc[i:end_ix, :])
                print(sequences.iloc[end_ix:(end_ix+n_outputs), -1])
            X.append(seq_x)
            y.append(seq_y)
    return np.array(X), np.array(y)

#Function that makes sure each input sequence has the same shape

def unique_shapes(x, y, lag_, n_features_, num_of_outputs_):
    uniuqe_shapes = []
    for k in range(len(x)):

        if (x[k].shape == (lag_, n_features_)) & (y[k].shape == (num_of_outputs_,)):
                uniuqe_shapes.append(k)

    x = x[uniuqe_shapes]
    y = y[uniuqe_shapes]
    x = np.stack(x)
    y = np.stack(y)
    return x, y

def scale_data(x_train, x_test, y_train, y_test):

    # Reshape features for scaling
    x_train_flat = x_train.reshape(-1, x_train.shape[-1])
    x_test_flat = x_test.reshape(-1, x_test.shape[-1])

    # Initialize scalers
    X_scaler = MinMaxScaler()
    Y_scaler = MinMaxScaler()

    # Fit scalers on training data
    X_scaler.fit(x_train_flat)
    Y_scaler.fit(y_train)

    # Scale features
    x_train_scaled = X_scaler.transform(x_train_flat).reshape(x_train.shape)
    x_test_scaled = X_scaler.transform(x_test_flat).reshape(x_test.shape)

    # Scale target
    y_train_scaled = Y_scaler.transform(y_train)
    y_test_scaled = Y_scaler.transform(y_test)

    return x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled, Y_scaler

def adjust_value(val, mean, lower_bound, upper_bound):
        if val < lower_bound:
            return mean + (val - lower_bound)/2  # Scale up the value closer to the mean
        elif val > upper_bound:
            return mean + (val - upper_bound)/2  # Scale down the value closer to the mean
        else:
            return val


def tweak_outliers(data, level, feature):
    while level > 0:
        for year in range (2016, 2018):
            for month in range(1,13):
                if month < 12:
                    value_list = data[(data['timestamp'] >= datetime(year, month, 1, 0, 0, 0)) & (data['timestamp'] <= datetime(year,month+1, 1, 0, 0, 0))][feature]
                    mean = value_list.mean()
                    std_dev = value_list.std()
                    lower_bound = mean - 2 * std_dev
                    upper_bound = mean + 2 * std_dev
                    for i in data.index:
                        if data.loc[i, 'timestamp'] >= datetime(year, month, 1, 0, 0, 0) and data.loc[i, 'timestamp'] < datetime(year, month+1, 1, 0, 0, 0) :
                            data.loc[i, feature] = adjust_value(data.loc[i, feature], mean, lower_bound, upper_bound)

                elif month == 12:
                    value_list = data[(data['timestamp'] >= datetime(year, month, 1, 0, 0, 0)) & (data['timestamp'] <= datetime(year+1, 1, 1, 0, 0, 0))][feature]
                    mean = value_list.mean()
                    std_dev = value_list.std()
                    lower_bound = mean - 2 * std_dev
                    upper_bound = mean + 2 * std_dev
                    for i in data.index:
                        if data.loc[i, 'timestamp'] >= datetime(year, month, 1, 0, 0, 0) and data.loc[i, 'timestamp'] < datetime(year+1, 1, 1, 0, 0, 0) :
                            data.loc[i, feature] = adjust_value(data.loc[i, feature], mean, lower_bound, upper_bound)


        level = level - 1

def fix_nans(data, feature):
    for i in range(len(data)):
        if math.isnan(data.loc[i, feature]):
            if i < 8784:
                data.loc[i, feature] = data_source[data_source["timestamp"]==data_source.loc[i, "timestamp"]+pd.DateOffset(years= 1)][feature].item()
            elif i >= 8784:
                data.loc[i, feature] = data_source[data_source["timestamp"]==data_source.loc[i, "timestamp"]-pd.DateOffset(years= 1)][feature].item()

            if math.isnan(data.loc[i, feature]):
                if i+24 < len(data_source):
                    data.loc[i, feature] = data_source[data_source["timestamp"]==data_source.loc[i, "timestamp"]+pd.DateOffset(days= 1)][feature].item()

                if i-24 >= 0 and math.isnan(data.loc[i, feature]):
                    data.loc[i, feature] = data_source[data_source["timestamp"]==data_source.loc[i, "timestamp"]-pd.DateOffset(days= 1)][feature].item()

                if math.isnan(data.loc[i, feature]):
                    if i+48 < len(data_source):
                        data.loc[i, feature] = data_source[data_source["timestamp"]==data_source.loc[i, "timestamp"]+pd.DateOffset(days= 2)][feature].item()

                    if i-48 >= 0 and math.isnan(data.loc[i, feature]):
                        data.loc[i, feature] = data_source[data_source["timestamp"]==data_source.loc[i, "timestamp"]-pd.DateOffset(days= 2)][feature].item()

                    if math.isnan(data.loc[i, feature]):
                        if i+72 < len(data_source):
                            data.loc[i, feature] = data_source[data_source["timestamp"]==data_source.loc[i, "timestamp"]+pd.DateOffset(days= 3)][feature].item()

                        if i-72 >= 0 and math.isnan(data.loc[i, feature]):
                            data.loc[i, feature] = data_source[data_source["timestamp"]==data_source.loc[i, "timestamp"]-pd.DateOffset(days= 3)][feature].item()

                        if math.isnan(data.loc[i, feature]):
                            if i+96 < len(data_source):
                                data.loc[i, feature] = data_source[data_source["timestamp"]==data_source.loc[i, "timestamp"]+pd.DateOffset(days= 4)][feature].item()

                            if i-96 >= 0 and math.isnan(data.loc[i, feature]):
                                data.loc[i, feature] = data_source[data_source["timestamp"]==data_source.loc[i, "timestamp"]-pd.DateOffset(days= 4)][feature].item()

                            if math.isnan(data.loc[i, feature]):
                                if i+120 < len(data_source):
                                    data.loc[i, feature] = data_source[data_source["timestamp"]==data_source.loc[i, "timestamp"]+pd.DateOffset(days= 5)][feature].item()

                                if i-120 >= 0 and math.isnan(data.loc[i, feature]):
                                    data.loc[i, feature] = data_source[data_source["timestamp"]==data_source.loc[i, "timestamp"]-pd.DateOffset(days= 5)][feature].item()



def plot_feature(data, feature, domain):
    plt.figure(figsize=(15, 9))
    plt.plot(data['timestamp'], data[feature], label='{} Dataset'.format(domain), color='green', linestyle=':')
    plt.xlabel('Timestamp')
    plt.ylabel(feature)
    plt.title('Visualization of {} across {} dataset'.format(feature, domain))
    plt.legend()
    plt.grid(True)

    # Display the plot
    plt.tight_layout()
    plt.show()


#convert timestamp to a datetime format

data_source['timestamp'] = pd.to_datetime(data_source['timestamp'])
data_target['timestamp'] = pd.to_datetime(data_target['timestamp'])
data_target_2['timestamp'] = pd.to_datetime(data_target_2['timestamp'])


#create a month column for each dataset

month_column = data_source['timestamp'].dt.month
data_source.insert(1, 'month', month_column)
data_source['month'] = data_source['month'].apply(strip_leading_zeros)

month_column = data_target['timestamp'].dt.month
data_target.insert(1, 'month', month_column)
data_target['month'] = data_target['month'].apply(strip_leading_zeros)

month_column = data_target_2['timestamp'].dt.month
data_target_2.insert(1, 'month', month_column)
data_target_2['month'] = data_target_2['month'].apply(strip_leading_zeros)


#--------------------------------------------------------------------------------------------------------#
#--------------------------------process the source dataset--------------------------------------------#
#--------------------------------------------------------------------------------------------------------#

print("NaN value streaks in data source")
print()


check_nan_intervals(data_source, 'NaN STREAK')


#plot before processing

plot_feature(data_source, "Electrical_Consumption", "source")


print("Filling NaN values in data source")
print()

for i in range(len(data_source)):
    if data_source.loc[i, 'Electrical_Consumption'] < 13:
        data_source.loc[i, 'Electrical_Consumption'] = data_source[data_source["timestamp"]==data_source.loc[i, "timestamp"]-pd.DateOffset(years= 1)]['Electrical_Consumption'].item()
    elif data_source.loc[i, 'Electrical_Consumption'] > 65:
        data_source.loc[i, 'Electrical_Consumption'] = (data_source.loc[data_source["timestamp"]==data_source.loc[i, "timestamp"]-pd.DateOffset(days= 1)]['Electrical_Consumption'].item() + data_source.loc[data_source["timestamp"]==data_source.loc[i, "timestamp"]+pd.DateOffset(days= 1)]['Electrical_Consumption'].item())/2



fix_nans(data_source, "cloudCoverage")
data_source["cloudCoverage"] = data_source["cloudCoverage"].interpolate(method='spline', order=3)
fix_nans(data_source, "airTemperature")
fix_nans(data_source, "dewTemperature")
fix_nans(data_source, "windDirection")
fix_nans(data_source, "windSpeed")

print("Fixing outliers in data source")
print()

tweak_outliers(data_source, 1, "Electrical_Consumption")
#tweak_outliers(data_target, 1, "cloudCoverage")
#tweak_outliers(data_target, 1, "airTemperature")
#tweak_outliers(data_target, 1, "dewTemperature")
#tweak_outliers(data_target, 1, "windDirection")
#tweak_outliers(data_target, 1, "windSpeed")



plot_feature(data_source, "Electrical_Consumption", "source")

check_nan_intervals(data_source, 'NaN STREAK')


#--------------------------------------------------------------------------------------------------------#
#--------------------------------process the target dataset----------------------------------------------#
#--------------------------------------------------------------------------------------------------------#

print("NaN value streaks in data target")
print()

check_nan_intervals(data_target, 'NaN STREAK')


#plot before processing

plot_feature(data_target, "Electrical_Consumption", "target")

print("Filling NaN values in data target")
print()


fix_nans(data_target, "Electrical_Consumption")



for i in range(len(data_target)):
    if data_target.loc[i,'timestamp'] >= datetime(2016, 1, 4, 0, 0, 0) and data_target.loc[i,'timestamp'] <= datetime(2017, 12, 27, 0, 0, 0):
        p1 = data_target[data_target['timestamp'] == data_target.loc[i, 'timestamp']-pd.DateOffset(days= 1)]['Electrical_Consumption'].item()
        p2 = data_target[data_target['timestamp'] == data_target.loc[i, 'timestamp']-pd.DateOffset(days= 2)]['Electrical_Consumption'].item()
        f1 = data_target[data_target['timestamp'] == data_target.loc[i, 'timestamp']+pd.DateOffset(days= 1)]['Electrical_Consumption'].item()
        f2 =  data_target[data_target['timestamp'] == data_target.loc[i, 'timestamp']+pd.DateOffset(days= 2)]['Electrical_Consumption'].item()

        if data_target.loc[i,'Electrical_Consumption'] >= max(p1,p2,f1,f2) + 8 or data_target.loc[i,'Electrical_Consumption'] <= min(p1,p2,f1,f2) - 8:
            data_target.loc[i,'Electrical_Consumption'] = (p1+p2+f1+f2)/4



for i in range(len(data_target)):
    if data_target.loc[i,'timestamp'] <= datetime(2016, 1, 10, 0, 0, 0) or (data_target.loc[i,'timestamp'] >= datetime(2016, 5, 8, 0, 0, 0) and data_target.loc[i,'timestamp'] <= datetime(2016, 8, 15, 0, 0, 0)):
        data_target.loc[i, 'Electrical_Consumption'] = data_target[data_target['timestamp'] == data_target.loc[i, 'timestamp']+pd.DateOffset(years= 1)]['Electrical_Consumption'].item()

    if data_target.loc[i,'timestamp'] >= datetime(2017, 12, 15, 0, 0, 0):
        data_target.loc[i, 'Electrical_Consumption'] = data_target[data_target['timestamp'] == data_target.loc[i, 'timestamp']-pd.DateOffset(years= 1)]['Electrical_Consumption'].item()



for i in range(len(data_target)):
    if data_target.loc[i,'Electrical_Consumption'] >= 120 or data_target.loc[i,'Electrical_Consumption'] <= 60:
        p1 = data_target[data_target['timestamp'] == data_target.loc[i, 'timestamp']-pd.DateOffset(days= 1)]['Electrical_Consumption'].item()
        p2 = data_target[data_target['timestamp'] == data_target.loc[i, 'timestamp']-pd.DateOffset(days= 2)]['Electrical_Consumption'].item()
        f1 = data_target[data_target['timestamp'] == data_target.loc[i, 'timestamp']+pd.DateOffset(days= 1)]['Electrical_Consumption'].item()
        f2 =  data_target[data_target['timestamp'] == data_target.loc[i, 'timestamp']+pd.DateOffset(days= 2)]['Electrical_Consumption'].item()

        data_target.loc[i,'Electrical_Consumption'] = (p1+p2+f1+f2)/4



fix_nans(data_target, "cloudCoverage")
fix_nans(data_target, "airTemperature")
fix_nans(data_target, "dewTemperature")
fix_nans(data_target, "windDirection")
fix_nans(data_target, "windSpeed")

print("Fixing outliers in data target")
print()


tweak_outliers(data_target, 1, "Electrical_Consumption")
#tweak_outliers(data_target, 1, "cloudCoverage")
#tweak_outliers(data_target, 1, "airTemperature")
#tweak_outliers(data_target, 1, "dewTemperature")
#tweak_outliers(data_target, 1, "windDirection")
#tweak_outliers(data_target, 1, "windSpeed")


#plot after the processing

plot_feature(data_target, "Electrical_Consumption", "target")


check_nan_intervals(data_target, 'NaN STREAK')

#--------------------------------------------------------------------------------------------------------#
#--------------------------------process the target 2 dataset----------------------------------------------#
#--------------------------------------------------------------------------------------------------------#


print("NaN value streaks in data target 2")
print()

check_nan_intervals(data_target_2, 'NaN STREAK')


#plot before the processing

plot_feature(data_target_2, "Electrical_Consumption", "source")


print("Filling NaN values in data target 2")
print()

for i in range(9860, 9864):
    next_day_value = data_target_2[data_target_2["timestamp"]==data_target_2.loc[i, "timestamp"]+pd.DateOffset(days= 1)]["Electrical_Consumption"].item()

    data_target_2.loc[i, 'Electrical_Consumption'] = next_day_value

for i in range(12860, 12865):
    last_year_value = data_target_2[data_target_2["timestamp"]==data_target_2.loc[i, "timestamp"]-pd.DateOffset(years= 1)]["Electrical_Consumption"].item()

    data_target_2.loc[i, 'Electrical_Consumption'] = last_year_value



#fixing NaN value on datapoint 10466
data_target_2.loc[10139, 'Electrical_Consumption'] = data_target_2[data_target_2["timestamp"]==data_target_2.loc[10139, "timestamp"]+pd.DateOffset(days= 1)]["Electrical_Consumption"].item()

#fixing the outlier on datapoint 12859
data_target_2.loc[10466, 'Electrical_Consumption'] = data_target_2[data_target_2["timestamp"]==data_target_2.loc[10466, "timestamp"]+pd.DateOffset(days= 1)]["Electrical_Consumption"].item()

for i in range(0, 3378):
    data_target_2.loc[i, 'Electrical_Consumption'] = data_target_2[data_target_2["timestamp"]==data_target_2.loc[i, "timestamp"]+pd.DateOffset(years= 1)]["Electrical_Consumption"].item()


for i in range(len(data_target_2)):
    if data_target_2.loc[i, 'Electrical_Consumption'] > 90 or data_target_2.loc[i, 'Electrical_Consumption'] < 35:
        if i > 8784 :
            data_target_2.loc[i, 'Electrical_Consumption'] = data_target_2[data_target_2["timestamp"]==data_target_2.loc[i, "timestamp"]-pd.DateOffset(years= 1)]["Electrical_Consumption"].item()

            if data_target_2.loc[i, 'Electrical_Consumption'] > 90 or data_target_2.loc[i, 'Electrical_Consumption'] < 35:
                data_target_2.loc[i, 'Electrical_Consumption'] = data_target_2[data_target_2["timestamp"]==data_target_2.loc[i, "timestamp"]-pd.DateOffset(days= 1)]["Electrical_Consumption"].item()

            if data_target_2.loc[i, 'Electrical_Consumption'] > 90 or data_target_2.loc[i, 'Electrical_Consumption'] < 35:
                data_target_2.loc[i, 'Electrical_Consumption'] = data_target_2[data_target_2["timestamp"]==data_target_2.loc[i, "timestamp"]-pd.DateOffset(days= 2)]["Electrical_Consumption"].item()

            if data_target_2.loc[i, 'Electrical_Consumption'] > 90 or data_target_2.loc[i, 'Electrical_Consumption'] < 35:
                data_target_2.loc[i, 'Electrical_Consumption'] = data_target_2[data_target_2["timestamp"]==data_target_2.loc[i, "timestamp"]-pd.DateOffset(days= 3)]["Electrical_Consumption"].item()
        elif i < 8784:
            data_target_2.loc[i, 'Electrical_Consumption'] = data_target_2[data_target_2["timestamp"]==data_target_2.loc[i, "timestamp"]+pd.DateOffset(years= 1)]["Electrical_Consumption"].item()

            if data_target_2.loc[i, 'Electrical_Consumption'] > 90 or data_target_2.loc[i, 'Electrical_Consumption'] < 35:
                data_target_2.loc[i, 'Electrical_Consumption'] = data_target_2[data_target_2["timestamp"]==data_target_2.loc[i, "timestamp"]+pd.DateOffset(days= 1)]["Electrical_Consumption"].item()

            if data_target_2.loc[i, 'Electrical_Consumption'] > 90 or data_target_2.loc[i, 'Electrical_Consumption'] < 35:
                data_target_2.loc[i, 'Electrical_Consumption'] = data_target_2[data_target_2["timestamp"]==data_target_2.loc[i, "timestamp"]+pd.DateOffset(days= 2)]["Electrical_Consumption"].item()

            if data_target_2.loc[i, 'Electrical_Consumption'] > 90 or data_target_2.loc[i, 'Electrical_Consumption'] < 35:
                data_target_2.loc[i, 'Electrical_Consumption'] = data_target_2[data_target_2["timestamp"]==data_target_2.loc[i, "timestamp"]+pd.DateOffset(days= 3)]["Electrical_Consumption"].item()




fix_nans(data_target_2, "cloudCoverage")
data_target_2["cloudCoverage"] = data_target_2["cloudCoverage"].interpolate(method='spline', order=3)
fix_nans(data_target_2, "airTemperature")
fix_nans(data_target_2, "dewTemperature")
fix_nans(data_target_2, "windDirection")
fix_nans(data_target_2, "windSpeed")

print("Fixing outliers in data target 2")
print()

tweak_outliers(data_target_2, 1, "Electrical_Consumption")
#tweak_outliers(data_target_2, 1, "cloudCoverage")
#tweak_outliers(data_target_2, 1, "airTemperature")
#tweak_outliers(data_target_2, 1, "dewTemperature")
#tweak_outliers(data_target_2, 1, "windDirection")
#tweak_outliers(data_target_2, 1, "windSpeed")

#plot after the processing

plot_feature(data_target_2, "Electrical_Consumption", "target_2")


check_nan_intervals(data_target_2, 'NaN STREAK')


#--------------------------------------------------------------------------------------------------------#
#-------------------------------------- Final preprocessing ---------------------------------------------#
#--------------------------------------------------------------------------------------------------------#

#create a new column representing the relationship between 2 previous ones

print("'month' one hot encoding, 'Electrical_Consumption/sqm' creation, sequence format creation and scaling for all datasets")
print()

extra_column = data_source['Electrical_Consumption'].copy()
extra_column2 = data_target['Electrical_Consumption'].copy()
extra_column3 = data_target_2['Electrical_Consumption'].copy()

sqm = data_source.at[1,'sqm']

with open('Domain sqms/sqm_source.json', 'w') as f:
    json.dump(sqm, f)

sqm2 = data_target.at[1,'sqm']

with open('Domain sqms/sqm_target.json', 'w') as f:
    json.dump(sqm2, f)

sqm3 = data_target_2.at[1,'sqm']

with open('Domain sqms/sqm_target_2.json', 'w') as f:
    json.dump(sqm3, f)

for i in range(max(len(data_source),len(data_target),len(data_target_2))):
    if i < len(data_source):
        extra_column[i] = (extra_column[i]/sqm)*100

    if i < len(data_target):
        extra_column2[i] = (extra_column2[i]/sqm2)*100

    if i < len(data_target_2):
        extra_column3[i] = (extra_column3[i]/sqm3)*100

data_source.insert(10, 'Electrical_Consumption/sqm', extra_column)
data_target.insert(10, 'Electrical_Consumption/sqm', extra_column2)
data_target_2.insert(10, 'Electrical_Consumption/sqm', extra_column3)

data_source.pop('sqm')
data_target.pop('sqm')
data_target_2.pop('sqm')

data_source.pop('Electrical_Consumption')
data_target.pop('Electrical_Consumption')
data_target_2.pop('Electrical_Consumption')

cols = ['timestamp',
        'month_1',
        'month_2',
        'month_3',
        'month_4',
        'month_5',
        'month_6',
        'month_7',
        'month_8',
        'month_9',
        'month_10',
        'month_11',
        'month_12',
        'lat',
        'lng',
        'yearbuilt',
        'airTemperature',
        'cloudCoverage',
        'dewTemperature',
        'windDirection',
        'windSpeed',
        'Electrical_Consumption/sqm']



# Splitting factor for training set and test set
split = 0.8

# Select the lag variable, the number of features (must be same with cols selected) and the horizon
lag = 48  #lookbackwindow
n_features = len(cols)
num_of_outputs = 1

split_starting_index_12M = data_source[data_source['timestamp'] == datetime(2016, 5, 1, 0, 0, 0)].index[0]
split_ending_index_12M = data_source[data_source['timestamp'] == datetime(2017, 5, 1, 0, 0, 0)].index[0]

split_starting_index_6M = data_source[data_source['timestamp'] == datetime(2016, 11, 1, 0, 0, 0)].index[0]
split_ending_index_6M = data_source[data_source['timestamp'] == datetime(2017, 5, 1, 0, 0, 0)].index[0]

timestamp_month_processing(data_source)
timestamp_month_processing(data_target)
timestamp_month_processing(data_target_2)

for i in range(2):

    if i==0:
        data_target_final = data_target.iloc[split_starting_index_12M: split_ending_index_12M]
        data_target_2_final = data_target_2.iloc[split_starting_index_12M: split_ending_index_12M]
    elif i==1:
        data_target_final = data_target.iloc[split_starting_index_6M: split_ending_index_6M]
        data_target_2_final = data_target_2.iloc[split_starting_index_6M: split_ending_index_6M]

    #train-test split the data
    train_source, test_source =  train_test_split(data_source)
    train_target, test_target =  train_test_split(data_target_final)
    train_target_2, test_target_2 =  train_test_split(data_target_2_final)


    # Create the sequential input : x(total number of sequences, lag, features), y(total number of sequences) for each dataset
    x_train_source_unscaled, y_train_source_unscaled = split_sequences(train_source, n_steps=lag, n_outputs=num_of_outputs)
    x_test_source_unscaled, y_test_source_unscaled = split_sequences(test_source, n_steps=lag, n_outputs=num_of_outputs)

    x_train_target_unscaled, y_train_target_unscaled = split_sequences(train_target, n_steps=lag, n_outputs=num_of_outputs)
    x_test_target_unscaled, y_test_target_unscaled = split_sequences(test_target, n_steps=lag, n_outputs=num_of_outputs)

    x_train_target_2_unscaled, y_train_target_2_unscaled = split_sequences(train_target_2, n_steps=lag, n_outputs=num_of_outputs)
    x_test_target_2_unscaled, y_test_target_2_unscaled = split_sequences(test_target_2, n_steps=lag, n_outputs=num_of_outputs)



    #source domain scaling

    if i == 0:

        #source domain scaling

        x_train_source, x_test_source, y_train_source, y_test_source, Y_scaler_source = scale_data(x_train_source_unscaled, x_test_source_unscaled, y_train_source_unscaled, y_test_source_unscaled)

        joblib.dump(Y_scaler_source, 'Scalers/12M/source_scaler.pkl')


        #target domain scaling

        x_train_target, x_test_target, y_train_target, y_test_target, Y_scaler_target = scale_data(x_train_target_unscaled, x_test_target_unscaled, y_train_target_unscaled, y_test_target_unscaled)

        joblib.dump(Y_scaler_target, 'Scalers/12M/target_scaler.pkl')

        #target 2 domain scaling

        x_train_target_2, x_test_target_2, y_train_target_2, y_test_target_2, Y_scaler_target_2 = scale_data(x_train_target_2_unscaled, x_test_target_2_unscaled, y_train_target_2_unscaled, y_test_target_2_unscaled)

        joblib.dump(Y_scaler_target_2, 'Scalers/12M/target_2_scaler.pkl')

    elif i == 1:

        #source domain scaling

        x_train_source, x_test_source, y_train_source, y_test_source, Y_scaler_source = scale_data(x_train_source_unscaled, x_test_source_unscaled, y_train_source_unscaled, y_test_source_unscaled)

        joblib.dump(Y_scaler_source, 'Scalers/6M/source_scaler.pkl')


        #target domain scaling

        x_train_target, x_test_target, y_train_target, y_test_target, Y_scaler_target = scale_data(x_train_target_unscaled, x_test_target_unscaled, y_train_target_unscaled, y_test_target_unscaled)

        joblib.dump(Y_scaler_target, 'Scalers/6M/target_scaler.pkl')

        #target 2 domain scaling

        x_train_target_2, x_test_target_2, y_train_target_2, y_test_target_2, Y_scaler_target_2 = scale_data(x_train_target_2_unscaled, x_test_target_2_unscaled, y_train_target_2_unscaled, y_test_target_2_unscaled)

        joblib.dump(Y_scaler_target_2, 'Scalers/6M/target_2_scaler.pkl')


    #Reshape the input so all sequences have same shape
    x_train_source, y_train_source = unique_shapes(x_train_source, y_train_source, lag, n_features, num_of_outputs)
    x_test_source, y_test_source = unique_shapes(x_test_source, y_test_source, lag, n_features, num_of_outputs)

    x_train_target, y_train_target = unique_shapes(x_train_target, y_train_target, lag, n_features, num_of_outputs)
    x_test_target, y_test_target = unique_shapes(x_test_target, y_test_target, lag, n_features, num_of_outputs)

    x_train_target_2, y_train_target_2 = unique_shapes(x_train_target_2, y_train_target_2, lag, n_features, num_of_outputs)
    x_test_target_2, y_test_target_2 = unique_shapes(x_test_target_2, y_test_target_2, lag, n_features, num_of_outputs)



    #Save preprocessed input files
    if i==0:

        np.save('Data/Preprocessed data/12M/source_train_x.npy', x_train_source)
        np.save('Data/Preprocessed data/12M/source_train_y.npy', y_train_source)
        np.save('Data/Preprocessed data/12M/source_test_x.npy', x_test_source)
        np.save('Data/Preprocessed data/12M/source_test_y.npy', y_test_source)

        np.save('Data/Preprocessed data/12M/target_train_x.npy', x_train_target)
        np.save('Data/Preprocessed data/12M/target_train_y.npy', y_train_target)
        np.save('Data/Preprocessed data/12M/target_test_x.npy', x_test_target)
        np.save('Data/Preprocessed data/12M/target_test_y.npy', y_test_target)

        np.save('Data/Preprocessed data/12M/target_2_train_x.npy', x_train_target_2)
        np.save('Data/Preprocessed data/12M/target_2_train_y.npy', y_train_target_2)
        np.save('Data/Preprocessed data/12M/target_2_test_x.npy', x_test_target_2)
        np.save('Data/Preprocessed data/12M/target_2_test_y.npy', y_test_target_2)

    elif i==1:

        np.save('Data/Preprocessed data/6M/source_train_x.npy', x_train_source)
        np.save('Data/Preprocessed data/6M/source_train_y.npy', y_train_source)
        np.save('Data/Preprocessed data/6M/source_test_x.npy', x_test_source)
        np.save('Data/Preprocessed data/6M/source_test_y.npy', y_test_source)

        np.save('Data/Preprocessed data/6M/target_train_x.npy', x_train_target)
        np.save('Data/Preprocessed data/6M/target_train_y.npy', y_train_target)
        np.save('Data/Preprocessed data/6M/target_test_x.npy', x_test_target)
        np.save('Data/Preprocessed data/6M/target_test_y.npy', y_test_target)

        np.save('Data/Preprocessed data/6M/target_2_train_x.npy', x_train_target_2)
        np.save('Data/Preprocessed data/6M/target_2_train_y.npy', y_train_target_2)
        np.save('Data/Preprocessed data/6M/target_2_test_x.npy', x_test_target_2)
        np.save('Data/Preprocessed data/6M/target_2_test_y.npy', y_test_target_2)
